In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import LSTM_Model
from pytorch_lightning import seed_everything

torch.manual_seed(1)

In [ ]:
seed_everything(42)
device = 'cpu'
early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=5, verbose=True, mode='auto')
model = OurModel().to(device)
trainer = Trainer(max_epochs=100, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,
                    progress_bar_refresh_rate=10, callbacks=[early_stop_callback])
trainer.tune(model)

trainer.fit(model)
save(model.state_dict(), 'Location of our saved model')